In [1]:
import os
import copy
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing, make_regression
from sklearn.model_selection import train_test_split
housing = fetch_california_housing()
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  

X_train, X_test, y_train, y_test = train_test_split(housing.data, housing.target, test_size=0.98, random_state=2)
XR, yR = make_regression(n_samples=100, n_features=2, noise=0, random_state=5)
X_train2, X_test2, y_train2, y_test2 = train_test_split(XR, yR, test_size=0.20, random_state=2)

def normalize_dataset(X):
    return tf.keras.utils.normalize(X)

c:\Users\Utente\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [17]:
class Layer(tf.Module):
    def __init__(self, activation=tf.identity):
        self.activation = activation
        self.build = False
        self.normalized = False

    @tf.function(reduce_retracing=True)
    def xavier_init(self, shape):
        in_dim, out_dim = shape
        xavier_lim = tf.sqrt(6.)/tf.sqrt(tf.cast(in_dim + out_dim, tf.float32))
        weight_vals = tf.cast(tf.random.uniform(shape=(in_dim, out_dim), 
                                        minval=-xavier_lim, maxval=xavier_lim, seed=22, dtype=tf.float32), dtype=tf.double)
        return weight_vals

    @tf.function
    def __call__(self, X):
        if not self.build:
            n_samples, n_features = X.shape
            self.weights = tf.Variable(self.xavier_init(shape=(n_features, n_samples)), name="Weights", dtype=tf.double, trainable=True, )
            self.bias = tf.Variable(tf.zeros(shape=1, dtype=tf.double), name="Bias", dtype=tf.double, trainable=True )
            self.build = True

        z = tf.add(tf.matmul(X, self.weights), self.bias)

        print(X.shape,"*",self.weights.shape,"+",self.bias.shape, "=", z.shape)
        return self.activation(z)

    

class MLPLinearRegressor(tf.Module):
    def __init__(self, layers,XR, X, XN, X_test, y, y_test, lr=0.001, epochs=10):
        self.layers = layers
        self.lr = lr
        self.epochs = epochs
        self.loaded = False
        self.X = X 
        self.dataset = XR
        self.XN = XN
        self.XN_test = normalize_dataset(X_test)
        self.y = y
        self.y_test = y_test
        self.X_test = X_test
        self.learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(lr, decay_steps=1000000, decay_rate=0.96, staircase=True)
        self.optimizer = tf.keras.optimizers.Adam(self.learning_rate)
        self.train_mse = tf.keras.metrics.MeanSquaredError()
        self.train_mae = tf.keras.metrics.MeanAbsoluteError()
        self.train_accuracy = tf.keras.metrics.MeanSquaredLogarithmicError()
        self.test_mse = tf.keras.metrics.MeanSquaredError()
        self.test_mae = tf.keras.metrics.MeanAbsoluteError()
        self.test_accuracy = tf.keras.metrics.MeanSquaredLogarithmicError()
        self.regularizer = tf.keras.layers.ActivityRegularization()
        self.loss_history = [e for e in range(epochs)]
        self.bias_history = [e for e in range(epochs)]
        self.mae_train_error_history = [e for e in range(epochs)]
        self.mse_train_error_history = [e for e in range(epochs)]
        self.mae_test_error_history = [e for e in range(epochs)]
        self.mse_test_error_history = [e for e in range(epochs)]
        self.predicted_train_history = [[ i for i in range(self.y.shape[0]) ] for e in range(epochs)]
        self.regloss_history = [e for e in range(epochs)]

    @tf.function(reduce_retracing=True)
    def train_accuracy_rrsse(self, predicted):
        return tf.divide(tf.sqrt(
            tf.divide(
                tf.reduce_sum(tf.square(tf.abs((tf.subtract(predicted, self.y))))),
                tf.reduce_sum(tf.square(tf.abs((tf.subtract(tf.reduce_mean(self.y), predicted)))))
            )), predicted.shape[0]
        )

    @tf.function(reduce_retracing=True)
    def calc_metrics(self):
        self.train_mse.reset_state()
        self.train_mae.reset_state()
        self.test_mse.reset_state()
        self.test_mae.reset_state()
        self.train_accuracy.update_state(self.y, self.predicted)
        self.train_mse.update_state(self.y, self.predicted)
        self.train_mae.update_state(self.y, self.predicted)
        self.test_accuracy.update_state(self.y_test, self.predicted_test)
        self.test_mse.update_state(self.y_test, self.predicted_test)
        self.test_mae.update_state(self.y_test, self.predicted_test)

    @tf.function(reduce_retracing=True)
    def train_step(self, X):
        for layer in self.layers:
            X = layer(X)
        return X

    def _predict_train(self, X, weights):
        compressed_weights = tf.reduce_mean(tf.subtract(tf.multiply(X, weights), self.bias), axis=1)
        return tf.reduce_mean(tf.add(tf.multiply(tf.transpose(X), compressed_weights), self.bias), axis=1)

    def predict_test(self):
        self.predicted_test = tf.reduce_mean(tf.add(tf.multiply(self.X_test, self.compressed_weights), self.bias), axis=1)

    def predict(self, x):
        return tf.reduce_mean(tf.add(tf.multiply(x, self.compressed_weights), self.bias))

    def calc_history(self, e, loss):
        self.train_mae.update_state(self.y, self.predicted)
        self.train_mse.update_state(self.y, self.predicted)
        self.test_mae.update_state(self.y_test, self.predicted_test)
        self.test_mse.update_state(self.y_test, self.predicted_test)
        self.bias_history[e] = self.bias
        self.loss_history[e] = loss.numpy()
        self.predicted_train_history[e] = self.predicted
        self.mae_train_error_history[e] = self.train_mae.result().numpy()
        self.mse_train_error_history[e] = self.train_mse.result().numpy()
        self.mae_test_error_history[e] = self.test_mae.result().numpy()
        self.mse_test_error_history[e] = self.test_mse.result().numpy()
    
    @tf.function
    def lasso(self):
        lassos = [w for w in range(len(self.layers))]
        for w in range(len(self.layers)):
            lassos[w] = tf.reduce_sum(tf.norm(self.layers[w].weights))
        return tf.reduce_mean(lassos)

    @tf.function
    def ridge(self):
        ridges = [w for w in range(len(self.layers))]
        for w in range(len(self.layers)):
            ridges[w] = tf.reduce_sum(tf.square(tf.norm(self.layers[w].weights)))
        return tf.reduce_mean(ridges)

    @tf.function(reduce_retracing=True)
    def loss(self, predicted):
        return tf.add(tf.losses.MSE(self.y, predicted), self.lasso())

    def run(self):
        for i, j in enumerate(self.XN.split(self.dataset)):
            self.weights = model.train_step(self.dataset[i])

        self.vars = [self.layers[0].weights, self.layers[1].weights, self.layers[2].weights,self.layers[3].weights,self.layers[4].weights, self.layers[0].bias, self.layers[1].bias, self.layers[2].bias,self.layers[3].bias,self.layers[4].bias]
        self.weights_list = [self.layers[0].weights, self.layers[1].weights, self.layers[2].weights,self.layers[3].weights,self.layers[4].weights,]

        for e in range(self.epochs):
            with tf.GradientTape(watch_accessed_variables=True, persistent=True) as tape:
                for i, train_index in enumerate(self.XN.split(self.dataset)):
                    print(i, train_index)
                    self.weights = self.train_step(self.XN)
                    self.bias = self.layers[len(self.layers)-1].bias
                    self.flatten_weights = tf.reduce_mean(self.weights, axis=0)
                    self.predicted = self._predict_train(tf.transpose(self.X), self.flatten_weights)
                    
                    loss = tf.nn.scale_regularization_loss(self.loss(self.predicted)) 

                    self.compressed_weights = tf.reduce_mean(tf.add(tf.multiply(tf.transpose(self.XN), tf.reduce_mean(self.weights, axis=0)), self.bias), axis=1)
                    self.predict_test()

                    self.calc_history(e, loss)

            grads = tape.gradient(loss, self.vars)
            self.optimizer.apply_gradients(zip(grads, self.vars)) 

model = MLPLinearRegressor([
    Layer(),
    Layer(),
    Layer(activation=tf.nn.relu),
    Layer(activation=tf.sigmoid),
    Layer()
],XR, Xk, XNk, X_test2, tf.squeeze(y_train2), tf.squeeze(y_test2), lr=0.01, epochs=120)


model.run()
model.calc_metrics()
print("Original: ", y_test2[0], "...Predicted: ", model.predict(X_test2[0]).numpy(), "...Tested:", model.predicted_test[0].numpy())

print("Train_MeanSquaredError: ", model.train_mse.result().numpy())
print("Train_MeanAbsoluteError: ", model.train_mae.result().numpy())
print("Train_MeanSquaredLogError: ", model.train_accuracy.result().numpy())
print('--------------------------------------------------------')
print("Test_MeanSquaredError: ", model.test_mse.result().numpy())
print("Train_MeanAbsoluteError: ", model.test_mae.result().numpy())
print("Test_MeanSquaredLogError: ", model.test_accuracy.result().numpy())

fig, ax = plt.subplots(1, 4, figsize=(20, 5))

ax[0].plot([e for e in range(model.epochs)], model.loss_history, label="Loss")
ax[0].plot([e for e in range(model.epochs)], tf.square(model.bias_history).numpy(), label="Bias")
ax[0].set_ylabel('Loss & Bias History')
ax[0].set_xlabel('Epochs')
ax[0].legend()

ax[1].plot([e for e in range(model.epochs)], model.mae_train_error_history, label="Train MAE")
ax[1].plot([e for e in range(model.epochs)], model.mae_test_error_history, label="Test MAE")
ax[1].plot([e for e in range(model.epochs)], model.mse_train_error_history, label="Train MSE")
ax[1].plot([e for e in range(model.epochs)], model.mse_test_error_history, label="Test MSE")
ax[1].set_ylabel('Metrics History')
ax[1].set_xlabel('Epochs')
ax[1].legend()

ax[2].scatter(X_train2[:, -1], y_train2[:])
ax[2].plot(X_train2[:, -1], model.predicted[:], color='r')
#ax[2].set_ylim([-200, 200])
#ax[2].set_xlim(-3,3)
ax[2].set_xlabel("Train")

ax[3].scatter(X_test2[:, -1], y_test2[:])
ax[3].plot(X_test2[:, -1], model.predicted_test[:], color='r')
#ax[3].set_ylim([-200, 200])
#ax[3].set_xlim(-3,3)
ax[3].set_xlabel("Test")

"""fig3d = plt.figure()
ax3d = fig3d.add_subplot(projection='3d')
ax3d.scatter(X_train2[:, 0], X_train2[:, 1], y_train2[:])
ax3d.plot(X_train2[:, 0], X_train2[:, 1], model.predicted[:])
"""

plt.show()



tf.Tensor(
[[ 7.63157356e-01 -3.33903513e-01]
 [ 3.68625643e-01 -4.78704138e-01]
 [ 4.63989054e+00  2.73544567e-01]
 [ 4.37465216e-01 -6.01878426e-03]
 [-5.64168816e-02  4.81583884e-01]
 [ 2.75573790e-02  3.46293520e-01]
 [ 3.13854988e-02 -5.52448581e-02]
 [ 6.38810304e-01  8.35957976e-02]
 [ 2.26540988e-01  9.75344934e-02]
 [-1.57187509e-02  3.81434312e-02]
 [ 2.30949782e-01 -2.57217869e-01]
 [ 1.22207423e+00  8.52512608e-02]
 [ 7.75335434e-01  7.04126915e-01]
 [ 2.71752685e-01 -3.33456044e-01]
 [ 1.18625596e-01  4.76278927e-02]
 [ 3.41435758e+00  1.40816846e+00]
 [ 5.10410288e+00 -4.30880482e-01]
 [-4.98817648e-02  2.39693001e-01]
 [ 2.08505616e+00  6.59597438e-01]
 [ 7.46805778e-02 -1.33370159e-01]
 [ 1.54747280e+00  9.65034575e-02]
 [ 1.89266328e+00  2.04019924e-01]
 [ 3.83494523e-01  5.06605095e-01]
 [ 5.87800272e-01  1.85970906e+00]
 [-1.51359085e-01  1.00360555e+00]
 [ 1.50619231e+00 -1.42749798e+00]
 [ 1.12030273e-01 -1.01318596e-01]
 [ 6.84250529e-02  1.68875889e-01]
 [ 6.2438